##Project 2 GAN

##### Carlos Santiago Castillo 
##### 164426


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import time
import keras
from keras import Input
from keras.layers import Dense, Reshape, LeakyReLU, Conv2D, Conv2DTranspose, Flatten, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tqdm import tqdm
from PIL import Image
from matplotlib import pyplot as plt
import imageio
import shutil
import keras.optimizers.schedules

In [ ]:
# Load CelebA dataset and split it into train and validation sets
celeba_builder = tfds.builder("celeb_a")
celeba_builder.download_and_prepare()
celeba_train = celeba_builder.as_dataset(split='train')
num_images = celeba_builder.info.splits['train'].num_examples

# Use only 50% of the images
num_images = int(num_images)
celeba_train = celeba_train.take(num_images)

In [ ]:
# Define generator model
generator = tf.keras.Sequential([
    tf.keras.layers.Dense(256, input_shape=(100000,), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(784, activation='tanh'),
    tf.keras.layers.Reshape((28, 28, 1))
])

# Define discriminator model
discriminator = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Define loss functions and optimizer
cross_entropy = tf.keras.losses.BinaryCrossentropy()
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Function to preprocess images
def preprocess_image(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    image = tf.image.resize(image, (28, 28))
    return image

# Function to generate batch of noise vectors
def generate_noise(batch_size, noise_dim):
    return np.random.normal(0, 1, size=(batch_size, noise_dim))

# Function to generate fake images from noise
def generate_fake_images(generator, noise_dim, num_images):
    noise = generate_noise(num_images, noise_dim)
    fake_images = generator.predict(noise)
    return fake_images

# Function to train the GAN
def train_gan(epochs, batch_size, noise_dim):
    # Define labels for real and fake images
    real_labels = tf.ones((batch_size, 1))
    fake_labels = tf.zeros((batch_size, 1))

    for epoch in range(epochs):
        for image_batch in celeba_train.batch(batch_size):
            # Preprocess images and generate noise
            images = preprocess_image(image_batch['image'])
            noise = generate_noise(batch_size, noise_dim)

            # Generate fake images using generator
            with tf.GradientTape() as gen_tape:
                fake_images = generator(noise)

                # Train generator on fake images
                gen_loss = cross_entropy(real_labels, discriminator(fake_images))
            gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))

            # Train discriminator on real and fake images
            with tf.GradientTape() as disc_tape:
                real_loss = cross_entropy(real_labels, discriminator(images))
                fake_loss = cross_entropy(fake_labels, discriminator(fake_images))
                disc_loss = real_loss + fake_loss
            disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

                # Print loss after each epoch
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}')

    # Generate and save sample images after every 100 epochs
    if epoch % 1000 == 0:
        fake_images = generate_fake_images(generator, noise_dim, 16)
        fig, axes = plt.subplots(4, 4, figsize=(8, 8), subplot_kw={'xticks': [], 'yticks': []})
        for i, ax in enumerate(axes.flat):
            ax.imshow(fake_images[i, :, :, 0], cmap='gray')
            ax.set_title(f'Image {i+1}')
        plt.savefig(f'gan_samples_epoch{epoch}.png')

In [ ]:
epochs = 100
iters = 10*epochs
batch_size = 16
noise_dim = 1000

train_gan(epochs, batch_size, noise_dim)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fake_images = generate_fake_images(generator, 100000, 1)
fig, axes = plt.subplots(4, 4, figsize=(8, 8), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(fake_images[i, :, :, 0])
    ax.set_title(f'Image {i+1}')
plt.show()